# Extract2Slice

This script:
1. Loops through subjects and loads in structural data
2. Warps and slices nmap
3. Extracts ROI data

## Load Packages

In [1]:
import os
import numpy as np
import pandas as pd
import nilearn
from nilearn import image as nimage
from nilearn.masking import apply_mask
import neuromaps
from neuromaps.images import (load_nifti, load_data)
from neuromaps.resampling import resample_images
import nibabel as nib
import os
import ants
from ants import image_read, apply_transforms
from nilearn.datasets import fetch_atlas_harvard_oxford

## Set variables

In [38]:
# Set path for downloading neuromaps data
root = '/Users/pecsok/Desktop/ImageData/PMACS_remote'

cest_path = root + '/data/cest'
#nmaps_path = ' /Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/fmri_pipeline/parcellated_pipeline/cest_nmaps_pipeline/'
#nmaps_path = ' /Users/Desktop/'
# nmaps_path = root + '/data/nmaps'
nmaps_path = '/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/maps/'
outpath = root + '/data/nmaps/sandbox'


structural_path = root + '/data/cest/preprocessed'
subjlist_csv = root + '/data/subject_list_031124.csv'
#subjlist = pd.read_csv(subjlist_csv)

nmaps = ["NMDA","GABA"] #,"mGluR5"
#nmaps = (root + '/data/nmaps/average_mGluR', root + '/data/nmaps/average_NMDA')

In [39]:
structural_path = '/Users/pecsok/Desktop'
outpath = '/Users/pecsok/Desktop'
nmaps_path = '/Users/pecsok/Desktop'
atlas = 'Schaefer2018_1000Parcels_17Networks'

native_nmaps = {}
sliced_nmaps = {}
extracted_nmaps = {}
case='20303_12234'

### Defines Functions

In [107]:
def warp_nmap(struct, nmap, xfm):
    # This function warps the nmap to subject space.
    mapnii = os.path.join(nmaps_path, 'resampled_' + nmap + '.nii.gz')
    nmap_native = apply_transforms(fixed=struct, moving=image_read(mapnii),
                                      transformlist=xfm['fwdtransforms'], interpolator='nearestNeighbor')
    # save it out
    outfile = os.path.join(outpath,case + nmap + '_nativespace.nii.gz')
    nmap_native.to_filename(outfile)
    return {nmap: outfile}
    

def slice_nmap(atlas, nmap, nmap_path):
    # Set paths
    atlas_path = os.path.join(structural_path, case, case + '-cest-' + atlas + '.nii.gz')
    outfile = os.path.join(outpath, case, case + '-sliced-' + nmap + '.nii.gz')
    print(outfile)
    # Create an identity transform in ITK format 
    identity_transform = sitk.Transform(3, sitk.sitkIdentity)
    # Apply SITK Transform
    vol_sitk = sitk.ReadImage(nmap_path)
    slice_sitk = sitk.ReadImage(atlas_path)
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(slice_sitk)
    resampler.SetTransform(identity_transform)
    resampled_vol = resampler.Execute(vol_sitk)
    # Save output
    sitk.WriteImage(resampled_vol, outfile) 
    return {nmap: outfile}


def extract_nmap(atlas, nmap, case, nmap_path):
    # Set empty vars
    avg_values = []
    num_voxels = []
    subject_dict = {'Subject': case} 
    print(subject_dict)
    subject_results = []
    
    # Load in data
    atlas_path = os.path.join(structural_path, case, case + '-cest-' + atlas + '.nii.gz')
    masknii = nib.load(atlas_path)
    nmapnii = nib.load(sliced_nmaps[nmap])
    mask_data = masknii.get_fdata()
    nmap_data = nmapnii.get_fdata()
    # Get parcel values and remove 0
    mask_values = np.unique(mask_data)
    mask_values = mask_values[mask_values != 0]
    
    # Loop over each unique value in the mask
    for value in mask_values:
        # Mask parcel
        section_mask = (mask_data == value)
        receptor_values = nmap_data[section_mask]
        # Average & num voxels
        avg_value = receptor_values.mean()
        num_voxel = np.sum(section_mask)
        subject_dict[f'{value}_Mean'] = avg_value
        subject_dict[f'{value}_Count'] = num_voxel
    # Save it out like a baller
    subject_results.append(subject_dict)
    results_df = pd.DataFrame(subject_results)
    return {nmap: results_df}
    

## Loop through subjects and extract nmaps slice data

In [42]:
# Loop through subjects
#for index, row in subjlist.iterrows():
#    bblid = str(int(row['BBLID']))
#    ses = str(int(row['SCANID_CEST']))
#    case = bblid + "_" + ses
#    print(case)

case='20303_12234'

# Read in structural data and make ants transform
struct = ants.image_read(os.path.join(structural_path, case, case + '-UNI_masked.nii.gz'))
struct_mni = ants.image_read(os.path.join(structural_path, case, 'MNI_transforms/', case + '-UNIinMNI-Warped.nii.gz'))
xfm = ants.registration(fixed=struct, moving=struct_mni,
                        type_of_transform='SyN', outprefix=outpath)
print("xfm created")


xfm created


In [109]:
for nmap in nmaps:        
    # Warp it
    native_nmap = warp_nmap(struct,nmap, xfm)
    native_nmaps.update(native_nmap) # Store path to native nmap
    # Slice it
    sliced_nmap = slice_nmap(atlas, nmap, native_nmaps[nmap])
    sliced_nmaps.update(sliced_nmap) # Store path to sliced nmap
    # Bop it (extract)
    extracted_nmap = extract_nmap(atlas, nmap, case, native_nmaps['NMDA'])
    extracted_nmaps.update(extracted_nmap)
    print(extracted_nmaps)


/Users/pecsok/Desktop/20303_12234/20303_12234-sliced-NMDA.nii.gz
{'Subject': '20303_12234'}
{'NMDA':        Subject  501.0_Mean  501.0_Count  502.0_Mean  502.0_Count  504.0_Mean  \
0  20303_12234    5.143796           40    7.742624           78    5.729826   

   504.0_Count  516.0_Mean  516.0_Count  518.0_Mean  ...  961.0_Mean  \
0           97    4.751892            1    5.956828  ...     7.04145   

   961.0_Count  962.0_Mean  962.0_Count  972.0_Mean  972.0_Count  973.0_Mean  \
0          137    6.871419           60    6.656556           37    7.679665   

   973.0_Count  974.0_Mean  974.0_Count  
0           58    7.951492            7  

[1 rows x 237 columns]}
/Users/pecsok/Desktop/20303_12234/20303_12234-sliced-GABA.nii.gz
{'Subject': '20303_12234'}
{'NMDA':        Subject  501.0_Mean  501.0_Count  502.0_Mean  502.0_Count  504.0_Mean  \
0  20303_12234    5.143796           40    7.742624           78    5.729826   

   504.0_Count  516.0_Mean  516.0_Count  518.0_Mean  ...  961